In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# 데이터 불러오기 (price 예측)
data = pd.read_csv("C://Users//Administrator//Downloads//05-11--machine-learning//Part 05~11) Machine Learning//08. Ensemble 기법의 종류와 원리//Data//kc_house_data.csv") 
data.head() # 데이터 확인


'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


## 필요없는 변수 제거
- 'id', 'date', 'zipcode', 'lat', 'long' 등을 제거한다.


In [8]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 독립,종속변수를 분리하고 , 학습,평가데이터 분리

In [21]:
X = data.iloc[:, data.columns != 'price']
y = data.iloc[:, data.columns == 'price']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 42) 
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape) 

(16209, 8) (5404, 8) (16209, 1) (5404, 1)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model_lr=LinearRegression()
model_lr.fit(X_train,y_train) 
predict_lr_test = model_lr.predict(X_test) 
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test,predict_lr_test)))) 

RMSE: 242752.385557006


## Bagging 을 이용하여 선형회귀모형에 적합 후 평가

In [31]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = model_lr,
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
model_lr_bagging = bagging_model.fit(X_train, y_train) 
model_bagging_predict = model_lr_bagging.predict(X_test)
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test,model_bagging_predict)))) 

RMSE: 242888.74361422364


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## Sampling 횟수를 늘림

In [32]:
bagging_model2 = BaggingRegressor(base_estimator = model_lr,
                                  n_estimators = 30, 
                                  verbose = 1) 
model_lr_bagging2 = bagging_model2.fit(X_train,y_train)
model_bagging_predict2 = model_lr_bagging2.predict(X_test)
print("RMSE: {}".format(sqrt(mean_squared_error(y_test,model_bagging_predict2 )))) # RMSE 결과

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 242831.52173720306


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 의사결정나무모형에 적합

In [36]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor() 
model_dt = dt_model.fit(X_train,y_train) 
model_dt_predict = model_dt.predict(X_test) 
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test,model_dt_predict)))) 

RMSE: 286660.49238936935


## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가

In [37]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = model_dt, 
                                                n_estimators = 5,
                                                verbose = 1) 
model_dt_bagging = bagging_decision_tree_model1.fit(X_train, y_train)
model_dt_predict_bagging = model_dt_bagging.predict(X_test)
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test,model_dt_predict_bagging)))) 

RMSE: 252427.69909130855


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [30]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = model_dt
                                                n_estimators = 30,
                                                verbose = 1)
model_dt_bagging2= bagging_decision_tree_model2.fit(X_train,y_train)
model_dt_predict_bagging2 = model_dt_bagging2(X_test)
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test,))))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 238589.21274712312


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
